# Residual networks
Implementation of  a simple residual network with 2 identity blocks

In [2]:
import torch
import torch.nn as nn

In [38]:
class ResidualBlock(nn.Module):
 def __init__(self, in_channels, out_channels):
    super(ResidualBlock, self).__init__()
    self.conv1 = nn.Sequential(
    nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
    nn.ReLU())
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1)
    self.relu = nn.ReLU()
    self.out_channels = out_channels
 def forward(self, x):
    residual = x
    out = self.conv1(x)
    out = self.conv2(out)
    out += residual
    out = self.relu(out)
    return out

In [60]:
class ResidualNetwork(nn.Module):
 def __init__(self):
    super(ResidualNetwork, self).__init__()
    self.conv1 = nn.Conv2d(3, 64, kernel_size = 7, stride =2, padding = 0)
    self.block1 = ResidualBlock(64, 64)
    self.block2 = ResidualBlock(64, 64)
    self.avg_pool = nn.AvgPool2d(kernel_size=2)
    self.fc = nn.Linear(64*54*54, 10)


 def forward(self, x):

    out = self.conv1(x)
    out = self.block1(out)
    out = self.block2(out)
    out = self.avg_pool(out)
    out = torch.flatten(out, 1)  # Flatten except batch dimension
    out = nn.Softmax(dim=1)(self.fc(out))
    return out

In [61]:

# Create model
model = ResidualNetwork()



In [62]:
from torchsummary import summary
summary(model, (3, 224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 109, 109]           9,472
            Conv2d-2         [-1, 64, 109, 109]          36,928
              ReLU-3         [-1, 64, 109, 109]               0
            Conv2d-4         [-1, 64, 109, 109]          36,928
              ReLU-5         [-1, 64, 109, 109]               0
     ResidualBlock-6         [-1, 64, 109, 109]               0
            Conv2d-7         [-1, 64, 109, 109]          36,928
              ReLU-8         [-1, 64, 109, 109]               0
            Conv2d-9         [-1, 64, 109, 109]          36,928
             ReLU-10         [-1, 64, 109, 109]               0
    ResidualBlock-11         [-1, 64, 109, 109]               0
        AvgPool2d-12           [-1, 64, 54, 54]               0
           Linear-13                   [-1, 10]       1,866,250
Total params: 2,023,434
Trainable param


# Multiple outputs
Simple multi-output convolutional network that predicts the age and gender of a person



In [63]:
import torch
import torch.nn as nn

# Define the common backbone (features)
class CommonBackbone(nn.Module):
    def __init__(self):
        super(CommonBackbone, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0)
        self.relu = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        return x

# Define age regression and gender classification heads
class AgeRegression(nn.Module):
    def __init__(self):
        super(AgeRegression, self).__init__()
        self.fc1 = nn.Linear(64 * 54 * 54, 32)  # Adjust input size based on your data dimensions after pooling
        self.relu = nn.ReLU()
        self.age_prediction = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        age_prediction = self.age_prediction(x)
        return age_prediction

class GenderClassification(nn.Module):
    def __init__(self):
        super(GenderClassification, self).__init__()
        self.fc2 = nn.Linear(64 * 54 * 54, 32)  # Adjust input size based on your data dimensions after pooling
        self.relu = nn.ReLU()
        self.gender_prediction = nn.Linear(32, 2)

    def forward(self, x):
        x = self.fc2(x)
        x = self.relu(x)
        gender_prediction = self.gender_prediction(x)
        return gender_prediction

# Create instances of backbone and heads
backbone = CommonBackbone()
age_head = AgeRegression()
gender_head = GenderClassification()

# Combine the model
class MultiTaskModel(nn.Module):
    def __init__(self, backbone, age_head, gender_head):
        super(MultiTaskModel, self).__init__()
        self.backbone = backbone
        self.age_head = age_head
        self.gender_head = gender_head

    def forward(self, x):
        x = self.backbone(x)
        age_output = self.age_head(x)
        gender_output = self.gender_head(x)
        return age_output, gender_output

# Construct the complete model
mt_model = MultiTaskModel(backbone, age_head, gender_head)

# Summary of the model
print(mt_model)


MultiTaskModel(
  (backbone): CommonBackbone(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (relu): ReLU()
    (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (age_head): AgeRegression(
    (fc1): Linear(in_features=186624, out_features=32, bias=True)
    (relu): ReLU()
    (age_prediction): Linear(in_features=32, out_features=1, bias=True)
  )
  (gender_head): GenderClassification(
    (fc2): Linear(in_features=186624, out_features=32, bias=True)
    (relu): ReLU()
    (gender_prediction): Linear(in_features=32, out_features=2, bias=True)
  )
)


In [66]:
from torchsummary import summary
summary(mt_model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 222, 222]             896
              ReLU-2         [-1, 32, 222, 222]               0
         MaxPool2d-3         [-1, 32, 111, 111]               0
            Conv2d-4         [-1, 64, 109, 109]          18,496
              ReLU-5         [-1, 64, 109, 109]               0
         MaxPool2d-6           [-1, 64, 54, 54]               0
           Flatten-7               [-1, 186624]               0
    CommonBackbone-8               [-1, 186624]               0
            Linear-9                   [-1, 32]       5,972,000
             ReLU-10                   [-1, 32]               0
           Linear-11                    [-1, 1]              33
    AgeRegression-12                    [-1, 1]               0
           Linear-13                   [-1, 32]       5,972,000
             ReLU-14                   

# Parameter sharing
Simple network which computes the sum of two inputs, preprocessed by a convolutional layer.



In [67]:
import torch
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        #without weight sharing
        self.conv_a = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv_b = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.add = nn.Sequential(nn.Conv2d(32, 32, kernel_size=1, stride=1, padding=0))  # To adjust channels if needed

    def forward(self, input_a, input_b):
        x_a = self.conv_a(input_a)
        x_a = self.relu(x_a)
        x_b = self.conv_b(input_b)
        x_b = self.relu(x_b)
        y = self.add(x_a + x_b)
        return y

# Assuming you have input tensors in PyTorch format
input_a = torch.randn(1, 3, 224, 224)  # Sample input for input_a
input_b = torch.randn(1, 3, 224, 224)  # Sample input for input_b

# Create an instance of the model
model = CustomModel()

# Pass the inputs through the model
output = model(input_a, input_b)

# Display a summary of the model
print(model)


CustomModel(
  (conv_a): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_b): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (add): Sequential(
    (0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  )
)


In [72]:
# compare weights of the two convolutional layers
import numpy as np
weights_a = model.conv_a.weight.data.numpy()
weights_b = model.conv_b.weight.data.numpy()
print(np.array_equal( weights_a, weights_b))

False


In [73]:
import torch
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv_a = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv_b = self.conv_a #re-use the same layer for weight sharing!

        self.relu = nn.ReLU()
        self.add = nn.Sequential(nn.Conv2d(32, 32, kernel_size=1, stride=1, padding=0))  # To adjust channels if needed

    def forward(self, input_a, input_b):
        x_a = self.conv_a(input_a)
        x_a = self.relu(x_a)
        x_b = self.conv_b(input_b)
        x_b = self.relu(x_b)
        y = self.add(x_a + x_b)
        return y

# Assuming you have input tensors in PyTorch format
input_a = torch.randn(1, 3, 224, 224)  # Sample input for input_a
input_b = torch.randn(1, 3, 224, 224)  # Sample input for input_b

# Create an instance of the model
model = CustomModel()

# Pass the inputs through the model
output = model(input_a, input_b)

# Display a summary of the model
print(model)

CustomModel(
  (conv_a): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_b): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (add): Sequential(
    (0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  )
)


In [80]:
# compare weights of the two convolutional layers
import numpy as np
weights_a = model.conv_a.weight.data.numpy()
weights_b = model.conv_b.weight.data.numpy()
print(np.array_equal( weights_a, weights_b))

True
